<a id="title_ID"></a>
# JWST Pipeline Validation Notebook: photom with MIRI Imager

<span style="color:red"> **Instruments Affected**</span>: MIRI, NIRCam, NIRISS, NIRSpec 

## Introduction

This test is designed to test the photom step in the calwebb_image2 pipeline. This notebook processes an image through the photom step of calwebb_image2 (calwebb_detector1 is optional) and examines the output image of the photom step. 

The documentation of the step describes it as follows:

The photom step applies flux (photometric) calibrations to a data product to convert the data from units of countrate to surface brightness. The calibration information is read from a photometric reference file. For imaging modes the PHOTOM reference file contains a table of exposure parameters that define various instrument configurations and the flux conversion data for each of those configurations. The table contains one row for each allowed combination of exposure parameters, such as detector and filter. The photom step searches the table for the row that matches the parameters of the science exposure and then loads the calibration information from that row of the table. The correction values are multiplied into the SCI and ERR arrays, and the square of the correction values are multiplied into the variance arrays.

The scalar conversion constant is copied to the header keyword PHOTMJSR, which gives the conversion from DN/s to megaJy/steradian that was applied to the data. The step also computes the equivalent conversion factor to units of microJy/square-arcsecond (or microjanskys) and stores it in the header keyword PHOTUJA2.

### Documentation

The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst


## Test description


The steps of the test in this notebook are as follow:

1) Read in data.

2) Run input data through calwebb_detector1. (if not reading in a rate or rateints file)

3) Run output of calwebb_detector1 through the photom step in calwebb_image2.

4) Get photom reference file. 

5) Compare the flat field reference file with the rate/cal image ratio and check that the correct conversion factor was applied.

6) Check that the pixel area information has been attached and that PIXAR_SR, PIXAR_A2, and BUNIT keywords have been populated in the cal image.



Author: T. Temim

In [ ]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)

In [ ]:
import os
if 'CRDS_CACHE_TYPE' in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']
print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

### Set up import statements

In [ ]:
import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from jwst.photom import PhotomStep
from jwst.datamodels import RampModel, ImageModel, MirImgPhotomModel, dqflags

from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from astropy import table

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from ci_watson.artifactory_helpers import get_bigdata
import crds


### Print pipeline version number

In [ ]:
jwst.__version__ 

### Read in data from artifactory

In [ ]:
input_file = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'flat_field',
                     'flat_field_miri_test', 
                     'car007_seq1_MIRIMAGE_F770Wexp1_b771_rate.fits')

### Read in input image as JWST data model

In [ ]:
from jwst import datamodels

im = ImageModel(input_file)

### Run input data through calwebb_detector1

In [ ]:
# The data read in is an image file already processed through Detector1. This step should only be run 
#if we're starting with a ramp file.

#det1 = Detector1Pipeline()
#det1.save_results = True
#det1.run(im)

### Run output of calwebb_detector1 through calwebb_image2 (or just the Photom step)

In [ ]:
#input_file = input_file.replace('rateint.fits', 'rate.fits)

In [ ]:
im2 = PhotomStep()
im2.save_results = True
im2.run(im)

In [ ]:
#im2 = Image2Pipeline()
#im2.background.skip = True
#im2.assign_wcs.skip = True
#im2.flat_field.skip = True
#im2.photom.skip=False
#im2.resample.skip = True
#im2.save_results = True
#im2.run(im)

In [ ]:
#input_file = input_file.replace('rate.fits', 'cal.fits')
input_file = input_file.replace('rate.fits', 'photomstep.fits')

In [ ]:
# read in the calibrated file
im_cal = ImageModel(input_file)

### Calculate the cal/rate image ratio

The ratio of the photom processed file divided by the rate file should equal the conversion factor read from the reference file (comparison below). Look at the ratio file.

In [ ]:
ratio_im = im_cal.data/im.data

print('min value of ratio image: ', np.nanmin(ratio_im))
print('max value of ratio image: ', np.nanmax(ratio_im))

In [ ]:
print(ratio_im)

In [ ]:
# Look at the ratio image
plt.figure(figsize=(20,20))

# mask out DO_NOT_USE values of 1
masked_ratio = np.ma.masked_where((im_cal.dq & dqflags.pixel['DO_NOT_USE'] > 0), ratio_im)

cmap = matplotlib.cm.get_cmap("Greys").copy()  # Can be any colormap that you want after the cm
cmap.set_bad(color='white') # color to mark all DO_NOT_USE pixels

plt.imshow(masked_ratio, cmap=cmap, origin='lower', vmin=0.55,vmax=0.65)
plt.colorbar()
plt.show()

### Get photom reference file

Look at the photom reference file being used and some parameters from the header.

In [ ]:
photomreffile = im_cal.meta.ref_file.photom.name
print('Photom reference file', photomreffile)
# find location of file
basename = crds.core.config.pop_crds_uri(photomreffile)
path = crds.locate_file(basename, "jwst")
# open reference file

photom_file = MirImgPhotomModel(path)

In [ ]:
# Print the contents of the phot table

photom_file.phot_table


Look at the value of the pixel area in steradians from the reference file

In [ ]:
photom_file.meta.photometry.pixelarea_steradians

Look at the value of the pixel area in arcseconds squared from the reference file

In [ ]:
photom_file.meta.photometry.pixelarea_arcsecsq

### Print photom calibration conversion factor for image

Look up the filter from the file header and the conversion factor from the table for that filter. Compare the value to that in the table printed above to be sure they match.

In [ ]:
print('The filter for this data is:', im_cal.meta.instrument.filter)

conv_factor = photom_file.phot_table['photmjsr'][np.where(photom_file.phot_table['filter'] == im_cal.meta.instrument.filter)][0]
print(conv_factor)

#print(photom_file.phot_table['photmjsr'][np.where(photom_file.phot_table['filter'] == im_cal.meta.instrument.filter)][0])

### Check that same value has been applied to cal image (three values should be the same)

The ratio image calculated previously should equal the value of the conversion factor from the table.

In [ ]:
print('image min','image max','ref file')
print(np.nanmin(ratio_im),np.nanmax(ratio_im),conv_factor)

try: 
    np.testing.assert_allclose(np.nanmin(ratio_im), conv_factor, rtol = 0.01)
except AssertionError: 
    print("AssertionError: The minimum value is not within 1% of the conversion factor")

try: 
    np.testing.assert_allclose(np.nanmax(ratio_im), conv_factor, rtol = 0.01)
except AssertionError: 
    print("AssertionError: The maximum value is not within 1% of the conversion factor")    

### Check that pixel area information has been attached

Check the extensions of the rate (input) and photom processed file to see if an AREA extension has been added.

In [ ]:
# Look at extensions of the rate file 
uncal_filename = str(im.meta.filename)
hdu = fits.open(uncal_filename)
hdu.info()
hdu.close()

In [ ]:
# Look at extensions of the rate file 
cal_filename = str(im_cal.meta.filename)
hdu = fits.open(cal_filename)
hdu.info()
hdu.close()

try:
    assert(im_cal.area.shape == im_cal.data.shape)
except AssertionError:
    print('AssertionError: AREA array is not the same shape as the data array')

#### Look at the pixel area extension that was attached

In [ ]:
# Look at stats of pixel area image 

print('pix_area_min', 'pix_area_max','pix_area_mean')
print(np.nanmin(im_cal.area),np.nanmax(im_cal.area),np.nanmean(im_cal.area))

In [ ]:
# Look at the area map image
plt.figure(figsize=(20,20))

# mask out DO_NOT_USE values of 1
masked_area = np.ma.masked_where((im_cal.dq & dqflags.pixel['DO_NOT_USE'] > 0), im_cal.area)

cmap = matplotlib.cm.get_cmap('rainbow').copy()  # Can be any colormap that you want after the cm
cmap.set_bad(color='white') # color to mark all DO_NOT_USE pixels

plt.imshow(masked_area, cmap=cmap, origin='lower', vmin=0.9,vmax=1.2)
plt.colorbar()
plt.show()

### Check that PIXAR_SR and PIXAR_A2 keywords have been populated

The header values PIXAR_SR and PIXAR_A2 values in the header of your output file should be populated and match the values from the photom reference file header.


In [ ]:
im_cal.find_fits_keyword('PIXAR_SR')

In [ ]:
im_cal.find_fits_keyword('PIXAR_A2')

#### Check that the header value from the output file matches the one from the reference file (PIXAR_SR)

In [ ]:
print(photom_file.meta.photometry.pixelarea_steradians,im_cal.meta.photometry.pixelarea_steradians)

try:
    assert photom_file.meta.photometry.pixelarea_steradians == im_cal.meta.photometry.pixelarea_steradians
except AssertionError:
    print('AssertionError: the header value PIXAR_SR in the output file header does not match the reference file.')

#### Check that the header value from the output file matches the one from the reference file (PIXAR_A2)

In [ ]:
print(photom_file.meta.photometry.pixelarea_arcsecsq, im_cal.meta.photometry.pixelarea_arcsecsq)

try:
    assert photom_file.meta.photometry.pixelarea_arcsecsq == im_cal.meta.photometry.pixelarea_arcsecsq
except AssertionError:
    print('AssertionError: the header value PIXAR_A2 in the output file header does not match the reference file.')

### Verify that pixel area in steradians is consistent with the pixel area in arcseconds squared

1 steradian = 206265^2 arcsec^2, so the area in arcseconds squared * the unit conversion should equal the area in steradians.

In [ ]:
print(im_cal.meta.photometry.pixelarea_arcsecsq/(206265.0**2),im_cal.meta.photometry.pixelarea_steradians)

converted_steradians = im_cal.meta.photometry.pixelarea_arcsecsq/(206265.0**2)


try:
     np.testing.assert_allclose(im_cal.meta.photometry.pixelarea_steradians, converted_steradians, rtol = 0.01)
except AssertionError:
    print('AssertionError: the two area units do not match.')

### Check that bunit information has been added to image ('MJy/sr')

Check that the units of the file have been updated to the correct ones, and that the numbers in the output file have changed to reflect the new units

In [ ]:
print('The units of the rate file are: ', im.meta.bunit_data)
print('The units of the photom processed file are: ', im_cal.meta.bunit_data)

In [ ]:
xval = 600
yval = 600

print('The value of the specified pixel in the rate file is: ', im.data[yval, xval])
print('The value of the specified pixel in the photom processed file is: ', im_cal.data[yval, xval])

print('The ratio between these values is: ', im_cal.data[yval, xval] / im.data[yval, xval])
print('The conversion factor from the table is: ', conv_factor)

try:
     np.testing.assert_allclose(im_cal.data[yval, xval] / im.data[yval, xval], conv_factor, rtol = 0.01)
except AssertionError:
    print('AssertionError: the science ratio does not match the conversion factor.')

### Check that the values of the ERR arrays also changed by the expected conversion factor

In [ ]:
xval = 600
yval = 600

print('The value of the specified pixel in the rate file is: ', im.err[yval, xval])
print('The value of the specified pixel in the photom processed file is: ', im_cal.err[yval, xval])

print('The ratio between these values is: ', im_cal.err[yval, xval] / im.err[yval, xval])
print('The conversion factor from the table is: ', conv_factor)

try:
     np.testing.assert_allclose(im_cal.err[yval, xval] / im.err[yval, xval], conv_factor, rtol = 0.01)
except AssertionError:
    print('AssertionError: the error ratio does not match the conversion factor.')

## Success criteria

If there are no reported AssertionError messages and the images displayed look reasonable, this test passes.


## About this Notebook
**Author:** T. Temim and M. Cracraft, INS/MIRI
<br>**Updated On:** 05/13/2021 